In [1]:
import tensorflow as tf
import scipy as sp
import numpy as np
import pandas as pd
import re
import hashlib
from sklearn.preprocessing import StandardScaler

import os
from pathlib import Path
import IPython.display as ipd
import cProfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
%matplotlib inline


from tensorflow.python.keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.utils import to_categorical

from sklearn.preprocessing import OrdinalEncoder

from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras.layers import Input, Dense, LSTM

from tensorflow.python.keras.layers import Embedding, Dropout, Activation

import sys
sys.path.insert(0, '../src/data')
import get_train_test

In [2]:
raw_train, raw_dev = get_train_test.get_train_test('../data/raw/train/audio/',10,12)

100 : 1.4626410103142606 s
200 : 1.6053225047476944 s
300 : 1.7623445178849304 s
400 : 1.9202329929160997 s
500 : 2.060560435101717 s
600 : 2.1993518766571802 s
700 : 2.3425187046230582 s
800 : 2.4827177364995845 s
900 : 2.6200289723195787 s
1000 : 3.1510527799190893 s
1100 : 3.680545920223967 s
1200 : 4.213536497861127 s
1300 : 4.738521738880713 s
1400 : 5.290853862914405 s
1500 : 5.824255927899867 s
1600 : 6.372203639878416 s
1700 : 6.930178432893715 s
1800 : 7.493927176995765 s


KeyboardInterrupt: 

In [34]:
raw_train[0][0].shape

(99, 161)

In [103]:
x_train = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_train])
x_dev = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_dev])

y_train = np.array([x[1] for x in raw_train])
y_dev = np.array([x[1] for x in raw_dev])

In [104]:
x_train.shape

(4882, 99, 161)

In [90]:
unk_train = np.where(y_train == 'unknown')[0]
unk_dev = np.where(y_dev == 'unknown')[0]

np.random.seed(42)
unk_train_downsampled = np.random.choice(unk_train, size=unk_train.shape[0] - int(x_train.shape[0]/27), replace=False)
unk_dev_downsampled = np.random.choice(unk_dev, size=unk_dev.shape[0] - int(x_dev.shape[0]/27), replace=False)

all_train_index = np.where(y_train)[0]
all_dev_index = np.where(y_dev)[0]

downsampled_train_index = np.setdiff1d(all_train_index,unk_train_downsampled)
downsampled_dev_index = np.setdiff1d(all_dev_index,unk_dev_downsampled)

x_train = x_train[downsampled_train_index]
x_dev = x_dev[downsampled_dev_index]

y_train = y_train[downsampled_train_index]
y_dev = y_dev[downsampled_dev_index]



In [105]:
labels_to_keep = ['yes', 'no', 'up', 'down', 'left',
                    'right', 'on', 'off', 'stop', 'go', 'silence']

for label in labels_to_keep + ['unknown']:
    print (f"{label} dev count: {len([x for x in np.array([x[1] for x in raw_dev ])[downsampled_dev_index] if x == label])}")

yes dev count: 18
no dev count: 19
up dev count: 20
down dev count: 23
left dev count: 21
right dev count: 17
on dev count: 23
off dev count: 21
stop dev count: 21
go dev count: 17
silence dev count: 0
unknown dev count: 18


In [37]:
class_weight = {0: 37,
                1: 37,
                2: 37,
                3: 37,
                4: 37,
                5: 37,
                6: 37,
                7: 37,
                8: 2,
                9: 37,
                10: 37}

In [106]:
x_resh = x_train.reshape(-1, 99*161)

scaler = StandardScaler()

scaler.fit(x_resh)

x_train_scaled = scaler.transform(x_resh).reshape(-1,99,161)
x_dev_scaled = scaler.transform(x_dev.reshape(-1,99*161)).reshape(-1,99,161)



In [110]:
np.random.seed(42)
rand_perm = np.random.permutation(x_train_scaled.shape[0])
x_train_scaled = x_train_scaled[rand_perm]
y_train_cat = y_train_cat[rand_perm]

In [111]:
from tensorflow.python.keras.layers import BatchNormalization

In [112]:
model = Sequential()
model.add(LSTM(110, return_sequences=True, input_shape=(99,161)))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(110, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(110))
model.add(BatchNormalization())
model.add(Dense(110))
model.add(BatchNormalization())
model.add(Dense(11))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

checkpointer = ModelCheckpoint(filepath='./model-{epoch:02d}.hdf5', verbose=1)

y_train_cat = to_categorical(OrdinalEncoder().fit_transform(y_train.reshape(-1,1)))
y_dev_cat = to_categorical(OrdinalEncoder().fit_transform(y_dev.reshape(-1,1)))

model.fit(x_train_scaled,y_train_cat, batch_size = 1024, epochs = 50,
                        validation_data=(x_dev_scaled,y_dev_cat), callbacks=[checkpointer],class_weight=class_weight)

Train on 4882 samples, validate on 512 samples
Epoch 1/50
4096/4882 [========================>.....] - ETA: 7s - loss: 2.9420 - categorical_accuracy: 0.0986 
Epoch 00001: saving model to ./model-01.hdf5
4882/4882 [==============================] - 60s 12ms/step - loss: 2.9014 - categorical_accuracy: 0.0965 - val_loss: 2.4034 - val_categorical_accuracy: 0.0605
Epoch 2/50
4096/4882 [========================>.....] - ETA: 5s - loss: 2.4471 - categorical_accuracy: 0.1011 
Epoch 00002: saving model to ./model-02.hdf5
4882/4882 [==============================] - 35s 7ms/step - loss: 2.4422 - categorical_accuracy: 0.0991 - val_loss: 2.4036 - val_categorical_accuracy: 0.0547
Epoch 3/50
4096/4882 [========================>.....] - ETA: 5s - loss: 2.2972 - categorical_accuracy: 0.1016 
Epoch 00003: saving model to ./model-03.hdf5
4882/4882 [==============================] - 33s 7ms/step - loss: 2.3003 - categorical_accuracy: 0.0989 - val_loss: 2.4004 - val_categorical_accuracy: 0.0840
Epoch 4/50

KeyboardInterrupt: 

In [109]:
x_pred = model.predict(x_dev_scaled)

x_best_pred = np.argmax(x_pred,axis=1)

pd.DataFrame(x_best_pred.reshape(-1,1))[0].value_counts()

8     403
2      56
7      14
10     11
4       9
9       7
1       6
3       3
6       1
5       1
0       1
Name: 0, dtype: int64